In [1]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:100% !important;}
div.cell.code_cell.rendered{width:100%;}
div.input_prompt{padding:0px;}
div.CodeMirror {font-family:Consolas; font-size:22pt;}
div.text_cell_render.rendered_html{font-size:18pt;}
div.text_cell_render.rendered_html{font-size:15pt;}
div.output {font-size:18pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:18pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:18pt;padding:5px;}
table.dataframe{font-size:18px;}
</style>
"""))

<span style="color:red; font-size: 45px; font-weith:bold;"> CH03.연관분석 </span>
# 1. 연관분석 개요
- 데이터들 사이에 자주 발생하는 속성을 찾고, 그 속성들 사이에 연관성이 어느정도 있는지 분석
- 활용분야 : 이벤트 미리감지(사기 적발,...), 신상품 카테고리 구성

[조건:left-hand side: 오렌지주스] -> [결과:right-hand side:와인]
- 연관분석과 관련된 지표

1. 지지도(support) : 얼마나 자주 함께 나타나는지에 대한 지표 (몇번나오는지?)\
    (lhs, rhs)의 항목수/전체항목수 = 0.2
<br>

2. 신뢰도(confidence) : 조건이 오면 결과가 얼마나 자주 나타나는지 (조건과 결과 중요)\
    (lhs → rhs)의 항목수lhs의 항목수 = 1/2 = 0.5  
    lhs → rhs 순서 중요
<br>

3. 향상도(lift) : 우연히 발생한 규칙이 아닌지 확인하는 지표\
    (lhs → rhs의 지지도 / lhs의 지지도*rhs의 지지도) = 0.2/(0.4*0.6=0.24) =0.833
    향상도 < 1 : 기대가 낮다.
    향상도 > 1 : 기대가 높다.
    
<br>


# 2. 연관분석 구현


In [5]:
import os
print(os.getcwd())

C:\ai_x\source\07_자연어처리\test


In [7]:
import csv
transaction=[]
with open('C:/ai_x/source/07_자연어처리/data/cf_basket.csv','r',encoding='utf-8')as f:
    csvdata = csv.reader(f)
    transaction = list(csvdata)
transaction

[['소주', '콜라', '와인'],
 ['소주', '오렌지주스', '콜라'],
 ['맥주', '콜라', '와인'],
 ['소주', '콜라', '맥주'],
 ['오렌지주스', '와인']]

In [12]:
from apyori import apriori
rules = apriori(transaction,         # 2차원 데이터
               min_support= 0.15,    #
               min_confidence=0.1)   #
# print(list(rules))
rules = list(rules)
len(rules)

18

In [17]:
rules[10]

RelationRecord(items=frozenset({'콜라', '소주'}), support=0.6, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'소주', '콜라'}), confidence=0.6, lift=1.0), OrderedStatistic(items_base=frozenset({'소주'}), items_add=frozenset({'콜라'}), confidence=1.0, lift=1.25), OrderedStatistic(items_base=frozenset({'콜라'}), items_add=frozenset({'소주'}), confidence=0.7499999999999999, lift=1.2499999999999998)])

In [24]:
rule = rules[10]
support = rule[1]
order_st = rule[2]
for item in order_st:
    lhs = item[0]
    rhs = item[1]
    confidence = item[2]
    lift = item[3]
    if lift > 1:
        print("{} =>{}\t {}\t {}\t {}".format(lhs,rhs,support,
                                              round(confidence,2),
                                              round(lift,2)))

frozenset({'소주'})=>frozenset({'콜라'})	 0.6	 1.0	 1.25
frozenset({'콜라'})=>frozenset({'소주'})	 0.6	 0.75	 1.25


In [31]:
for rule in rules:
    support = rule[1]
    order_st = rule[2]
    for item in order_st:
        lhs = ', '.join([data for data in item[0]])
        rhs = ', '.join([data for data in item[1]])
        confidence = item[2]
        lift = item[3]
        if lift > 1:
            print("{} =>{}\t {}\t {}\t {}".format(lhs,rhs,support,
                                                  round(confidence,2),
                                                  round(lift,2)))

맥주 =>콜라	 0.4	 1.0	 1.25
콜라 =>맥주	 0.4	 0.5	 1.25
소주 =>콜라	 0.6	 1.0	 1.25
콜라 =>소주	 0.6	 0.75	 1.25
콜라 =>맥주, 소주	 0.2	 0.25	 1.25
맥주, 소주 =>콜라	 0.2	 1.0	 1.25
맥주 =>와인, 콜라	 0.2	 0.5	 1.25
콜라 =>와인, 맥주	 0.2	 0.25	 1.25
와인, 맥주 =>콜라	 0.2	 1.0	 1.25
와인, 콜라 =>맥주	 0.2	 0.5	 1.25
소주 =>오렌지주스, 콜라	 0.2	 0.33	 1.67
콜라 =>오렌지주스, 소주	 0.2	 0.25	 1.25
오렌지주스, 소주 =>콜라	 0.2	 1.0	 1.25
오렌지주스, 콜라 =>소주	 0.2	 1.0	 1.67
콜라 =>와인, 소주	 0.2	 0.25	 1.25
와인, 소주 =>콜라	 0.2	 1.0	 1.25


In [34]:
import pandas as pd
rules_df = pd.DataFrame(None, columns=['lhs','rhs','지지도','신뢰도','향상도'])
idx=0
for rule in rules:
    support = rule[1]
    order_st = rule[2]
    for item in order_st:
        lhs = ', '.join([data for data in item[0]])
        rhs = ', '.join([data for data in item[1]])
        confidence = item[2]
        lift = item[3]
        if lift > 1:
            rules_df.loc[idx] = [lhs,rhs,support,round(confidence,2), round(lift,2)]
            idx += 1
rules_df.sort_values(by=['향상도','신뢰도'],ascending=False)

,lhs,rhs,지지도,신뢰도,향상도
13,"오렌지주스, 콜라",소주,0.2,1.00,1.67
10,소주,"오렌지주스, 콜라",0.2,0.33,1.67
0,맥주,콜라,0.4,1.00,1.25
2,소주,콜라,0.6,1.00,1.25
5,"맥주, 소주",콜라,0.2,1.00,1.25
8,"와인, 맥주",콜라,0.2,1.00,1.25
12,"오렌지주스, 소주",콜라,0.2,1.00,1.25
15,"와인, 소주",콜라,0.2,1.00,1.25
3,콜라,소주,0.6,0.75,1.25
1,콜라,맥주,0.4,0.50,1.25


# 3. 전주/경주 여행 자료 연관분석

In [55]:
import pandas as pd
from konlpy.tag import Hannanum,Kkma,Komoran
df1 = pd.read_csv(r'C:/ai_x/source/07_자연어처리/data/naver_jj.csv', sep='\t')
df2 = pd.read_csv(r'C:/ai_x/source/07_자연어처리/data/naver_gj.csv', sep='\t')
jj_text_list = df1['jj_text'].tolist()
gj_text_list = df2['gj_text'].tolist()

analyzer = Komoran()
jj_noun_list = []
# select_pos = ['NC','NQ']  # Hannanum 보통명사, 고유명사
select_pos = ['NNG','NNP'] # 보통명사, 고유명사
불용어 = {'여행'}
for jj_text in jj_text_list:
#     jj_noun = analyzer.nouns(jj_text)
    jj_noun = [token for token, tag in analyzer.pos(jj_text) if tag in select_pos and token not in 불용어 and len(token) >1]
    jj_noun_list.append(jj_noun)
jj_noun_list[:2]

[['전주',
  '한곳',
  '추천',
  '추억',
  '체험',
  '감성',
  '추억',
  '테마',
  '박물관',
  '시간',
  '전북',
  '전북',
  '투어',
  '패스',
  '통합',
  '이용',
  '여행지',
  '체험',
  '카페',
  '이용',
  '추가',
  '전주',
  '필수',
  '편안',
  '감사'],
 ['전주',
  '전주',
  '사람과',
  '호텔',
  '가격',
  '얼마',
  '정도',
  '맛집',
  '카페',
  '추천',
  '안녕하세요',
  '전주',
  '계획',
  '한옥마을',
  '근처']]

In [68]:
%%time
rules = apriori(jj_noun_list, min_support=0.15, min_confidence=0.3)
rules = list(rules)
len(rules)

CPU times: total: 172 ms
Wall time: 166 ms


529

In [70]:

rules_df = pd.DataFrame(None, columns=['lhs','rhs','지지도','신뢰도','향상도'])
idx=0
for rule in rules:
    support = rule[1]
    order_st = rule[2]
    for item in order_st:
        lhs = ', '.join([data for data in item[0]])
        rhs = ', '.join([data for data in item[1]])
        confidence = item[2]
        lift = item[3]
        if lift > 1:
            rules_df.loc[idx] = [lhs,rhs,support,round(confidence,2), round(lift,2)]
            idx += 1

rules_df.sort_values(by=['향상도','신뢰도'],ascending=False, inplace=True)
rules_df = rules_df.reset_index(drop=True)

KeyboardInterrupt: 

In [62]:
pd.options.display.max_rows

60

In [66]:
rules_df.head(60)

,lhs,rhs,지지도,신뢰도,향상도
0,"국내, 전문",제주도,0.23,1.0,3.33
1,"전문, 맛집",제주도,0.24,1.0,3.33
2,"여수, 전문",제주도,0.24,1.0,3.33
3,"전문, 주가",제주도,0.24,1.0,3.33
4,"전문, 한곳",제주도,0.24,1.0,3.33
5,"국내, 전문","제주도, 맛집",0.23,1.0,3.33
6,"국내, 전문, 맛집",제주도,0.23,1.0,3.33
7,"국내, 전문","제주도, 여수",0.23,1.0,3.33
8,"여수, 국내, 전문",제주도,0.23,1.0,3.33
9,"국내, 전문","제주도, 전주",0.23,1.0,3.33
